In [1]:

from pymodbus.client import ModbusSerialClient
import time
import csv

# ✅ Configure Modbus RTU Client
client = ModbusSerialClient(
    port="COM7",  # Change based on your system
    baudrate=19200,
    stopbits=1,
    bytesize=8,
    parity="N",
    timeout=3
)

# ✅ Register Addresses (Subtract 1 for pymodbus)
REGISTERS = {
    "Z-Axis RMS Velocity (in/sec)": 45201 - 40001,
    "Z-Axis RMS Velocity (mm/sec)": 45202 - 40001,
    "Temperature (°F)": 45203 - 40001,
    "Temperature (°C)": 45204 - 40001,
    "X-Axis RMS Velocity (in/sec)": 45205 - 40001,
    "X-Axis RMS Velocity (mm/sec)": 45206 - 40001,
    "Z-Axis Peak Acceleration (G)": 45207 - 40001,
    "X-Axis Peak Acceleration (G)": 45208 - 40001,
    "Z-Axis Peak Velocity (in/sec)": 45217 - 40001,
    "Z-Axis Peak Velocity (mm/sec)": 45218 - 40001,
    "X-Axis Peak Velocity (in/sec)": 45219 - 40001,
    "X-Axis Peak Velocity (mm/sec)": 45220 - 40001,
    "Z-Axis High-Frequency RMS Acceleration (G)": 45221 - 40001,
    "X-Axis High-Frequency RMS Acceleration (G)": 45222 - 40001,
}

# ✅ Scaling Factors (Based on Data)
SCALING_FACTORS = {
    "Z-Axis RMS Velocity (in/sec)": 6.5535 / 65535,
    "Z-Axis RMS Velocity (mm/sec)": 65.535 / 65535,
    "Temperature (°F)": 327.67 / 32767,
    "Temperature (°C)": 327.67 / 32767,
    "X-Axis RMS Velocity (in/sec)": 6.5535 / 65535,
    "X-Axis RMS Velocity (mm/sec)": 65.535 / 65535,
    "Z-Axis Peak Acceleration (G)": 65.535 / 65535,
    "X-Axis Peak Acceleration (G)": 65.535 / 65535,
    "Z-Axis Peak Velocity (in/sec)": 6.5535 / 65535,
    "Z-Axis Peak Velocity (mm/sec)": 65.535 / 65535,
    "X-Axis Peak Velocity (in/sec)": 6.5535 / 65535,
    "X-Axis Peak Velocity (mm/sec)": 65.535 / 65535,
    "Z-Axis High-Frequency RMS Acceleration (G)": 65.535 / 65535,
    "X-Axis High-Frequency RMS Acceleration (G)": 65.535 / 65535,
}

# ✅ Create CSV File
DATA_FILE = "vibration_data.csv"

with open(DATA_FILE, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Timestamp"] + list(REGISTERS.keys()))  # CSV Headers

# ✅ Function to Read Modbus Register
def read_register(address, scale_factor):
    """Reads Modbus register and applies scaling factor."""
    try:
        response = client.read_holding_registers(address=address, count=1, slave=1)
        if response and response.registers:
            raw_value = response.registers[0]
            return round(raw_value * scale_factor, 3)
        else:
            return None
    except Exception as e:
        print(f"Error reading register {address}: {e}")
        return None

# ✅ Start Continuous Data Collection
try:
    if client.connect():
        print("✅ Connected to Modbus RTU device")
    else:
        print("❌ Failed to connect")
        exit()

    while True:
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        data_row = [timestamp]

        for param, reg in REGISTERS.items():
            value = read_register(reg, SCALING_FACTORS[param])
            data_row.append(value)

        # ✅ Save data to CSV
        with open(DATA_FILE, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(data_row)

    
        print(f"📊 Data Saved: {data_row}")

        time.sleep(1)  # Adjust based on data collection frequency

except KeyboardInterrupt:
    print("\n📌 Data collection stopped.")
finally:
    client.close()


✅ Connected to Modbus RTU device
📊 Data Saved: ['2025-02-19 15:27:17', 0.026, 0.662, 93.07, 33.92, 0.011, 0.273, 0.037, 0.038, 0.037, 0.937, 0.015, 0.386, 0.01, 0.01]
📊 Data Saved: ['2025-02-19 15:27:18', 0.026, 0.662, 93.07, 33.92, 0.011, 0.273, 0.037, 0.038, 0.037, 0.937, 0.015, 0.386, 0.01, 0.01]
📊 Data Saved: ['2025-02-19 15:27:20', 0.025, 0.649, 93.02, 33.9, 0.01, 0.253, 0.042, 0.035, 0.036, 0.917, 0.014, 0.358, 0.009, 0.01]

📌 Data collection stopped.


In [5]:
#this data collection is for OEE calculation model
import pandas as pd
import numpy as np
from pymodbus.client import ModbusSerialClient
import time
import csv

# ✅ Configure Modbus RTU Client
client = ModbusSerialClient(
    port="COM7",  # Change based on your system
    baudrate=19200,
    stopbits=1,
    bytesize=8,
    parity="N",
    timeout=3
)

# ✅ Register Addresses and Scaling Factors (same as before)
REGISTERS = {
    "Z-Axis RMS Velocity (in/sec)": 45201 - 40001,
    "Z-Axis RMS Velocity (mm/sec)": 45202 - 40001,
    "Temperature (°F)": 45203 - 40001,
    "Temperature (°C)": 45204 - 40001,
    "X-Axis RMS Velocity (in/sec)": 45205 - 40001,
    "X-Axis RMS Velocity (mm/sec)": 45206 - 40001,
    "Z-Axis Peak Acceleration (G)": 45207 - 40001,
    "X-Axis Peak Acceleration (G)": 45208 - 40001,
    "Z-Axis Peak Velocity (in/sec)": 45217 - 40001,
    "Z-Axis Peak Velocity (mm/sec)": 45218 - 40001,
    "X-Axis Peak Velocity (in/sec)": 45219 - 40001,
    "X-Axis Peak Velocity (mm/sec)": 45220 - 40001,
    "Z-Axis High-Frequency RMS Acceleration (G)": 45221 - 40001,
    "X-Axis High-Frequency RMS Acceleration (G)": 45222 - 40001,
}

SCALING_FACTORS = {
    "Z-Axis RMS Velocity (in/sec)": 6.5535 / 65535,
    "Z-Axis RMS Velocity (mm/sec)": 65.535 / 65535,
    "Temperature (°F)": 327.67 / 32767,
    "Temperature (°C)": 327.67 / 32767,
    "X-Axis RMS Velocity (in/sec)": 6.5535 / 65535,
    "X-Axis RMS Velocity (mm/sec)": 65.535 / 65535,
    "Z-Axis Peak Acceleration (G)": 65.535 / 65535,
    "X-Axis Peak Acceleration (G)": 65.535 / 65535,
    "Z-Axis Peak Velocity (in/sec)": 6.5535 / 65535,
    "Z-Axis Peak Velocity (mm/sec)": 65.535 / 65535,
    "X-Axis Peak Velocity (in/sec)": 6.5535 / 65535,
    "X-Axis Peak Velocity (mm/sec)": 65.535 / 65535,
    "Z-Axis High-Frequency RMS Acceleration (G)": 65.535 / 65535,
    "X-Axis High-Frequency RMS Acceleration (G)": 65.535 / 65535,
}

# ✅ Create CSV File
DATA_FILE = "vibration_data_with_OEE.csv"

# Create CSV header for the data
with open(DATA_FILE, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Timestamp"] + list(REGISTERS.keys()) + ["Availability", "Performance", "Quality", "OEE"])

# ✅ Function to Read Modbus Register
def read_register(address, scale_factor):
    """Reads Modbus register and applies scaling factor."""
    try:
        response = client.read_holding_registers(address=address, count=1, slave=1)
        if response and response.registers:
            raw_value = response.registers[0]
            return round(raw_value * scale_factor, 3)
        else:
            return None
    except Exception as e:
        print(f"Error reading register {address}: {e}")
        return None

# ✅ OEE Calculation Function
def calculate_oee(sensor_data, planned_production_time, ideal_speed, target_vibration_threshold):
    # Availability = Actual Production Time / Planned Production Time
    actual_production_time = len(sensor_data)  # Assuming each timestamp is a production time
    availability = actual_production_time / planned_production_time
    
    # Performance = Actual Speed / Ideal Speed
    actual_speed = np.mean(sensor_data['Z-Axis RMS Velocity (in/sec)'])  # Avg speed when running
    performance = actual_speed / ideal_speed
    
    # Quality = Good Output / Total Output
    quality_status = (sensor_data['Z-Axis RMS Velocity (in/sec)'] < target_vibration_threshold).astype(int)
    good_output = quality_status.sum()
    total_output = len(sensor_data)
    quality = good_output / total_output
    
    # OEE = Availability * Performance * Quality
    oee = availability * performance * quality
    
    return availability, performance, quality, oee

# ✅ Collect Data and Generate OEE
def collect_data_for_oee():
    # Placeholder for collecting data, where we'll save the OEE as well
    sensor_data = []
    
    # Connect to Modbus RTU
    if client.connect():
        print("✅ Connected to Modbus RTU device")
    else:
        print("❌ Failed to connect")
        return
    
    planned_production_time = 5000  # Planned production time in data frame number
    ideal_speed = 0.025  # Ideal speed (vibration level in in/sec)
    target_vibration_threshold = 0.030  # Threshold for good quality
    
    while True:
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        data_row = [timestamp]
        
        # Collect sensor data
        for param, reg in REGISTERS.items():
            value = read_register(reg, SCALING_FACTORS[param])
            data_row.append(value)
        
        # Add data to sensor_data for OEE calculation
        sensor_data.append({
            "timestamp": timestamp,
            **{param: data_row[idx + 1] for idx, param in enumerate(REGISTERS.keys())}
        })
        
        # Convert to DataFrame for OEE calculation
        sensor_df = pd.DataFrame(sensor_data)
        
        # Calculate OEE
        availability, performance, quality, oee = calculate_oee(sensor_df, planned_production_time, ideal_speed, target_vibration_threshold)
        
        # Save data to CSV
        data_row += [availability, performance, quality, oee]
        with open(DATA_FILE, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(data_row)
        
        print(f"📊 Data Saved: {data_row}")
        time.sleep(1)  # Adjust based on data collection frequency

# Start data collection
collect_data_for_oee()

client.close()

✅ Connected to Modbus RTU device
📊 Data Saved: ['2025-02-19 15:31:36', 0.026, 0.655, 93.07, 33.92, 0.013, 0.333, 0.038, 0.043, 0.036, 0.926, 0.018, 0.472, 0.01, 0.01, 0.0002, 1.0399999999999998, 1.0, 0.00020799999999999996]
📊 Data Saved: ['2025-02-19 15:31:37', 0.026, 0.655, 93.07, 33.92, 0.013, 0.333, 0.038, 0.043, 0.036, 0.926, 0.018, 0.472, 0.01, 0.01, 0.0004, 1.0399999999999998, 1.0, 0.0004159999999999999]
📊 Data Saved: ['2025-02-19 15:31:39', 0.025, 0.628, 93.07, 33.92, 0.011, 0.288, 0.046, 0.039, 0.035, 0.888, 0.016, 0.407, 0.01, 0.01, 0.0006, 1.0266666666666666, 1.0, 0.0006159999999999999]
📊 Data Saved: ['2025-02-19 15:31:40', 0.025, 0.628, 92.98, 33.87, 0.011, 0.288, 0.046, 0.039, 0.035, 0.888, 0.016, 0.407, 0.01, 0.01, 0.0008, 1.02, 1.0, 0.0008160000000000001]
📊 Data Saved: ['2025-02-19 15:31:42', 0.025, 0.628, 92.98, 33.87, 0.011, 0.288, 0.046, 0.039, 0.035, 0.888, 0.016, 0.407, 0.01, 0.01, 0.001, 1.0159999999999998, 1.0, 0.0010159999999999998]
📊 Data Saved: ['2025-02-19 15:3

KeyboardInterrupt: 

In [8]:
pip install chardet


Note: you may need to restart the kernel to use updated packages.


In [9]:
import chardet

def detect_encoding(file_path):
    with open(file_path, "rb") as f:
        result = chardet.detect(f.read())
        return result["encoding"]

# Detect encoding of each file
for file in ["vibration_data1.csv", "vibration_data2.csv", "vibration_data3.csv"]:
    encoding = detect_encoding(file)
    print(f"File: {file} | Encoding: {encoding}")


File: vibration_data1.csv | Encoding: ISO-8859-1
File: vibration_data2.csv | Encoding: ISO-8859-1
File: vibration_data3.csv | Encoding: ISO-8859-1


In [10]:
import pandas as pd

# Read CSV with a different encoding
df1 = pd.read_csv("vibration_data1.csv", encoding="ISO-8859-1")
df2 = pd.read_csv("vibration_data2.csv", encoding="ISO-8859-1")
df3 = pd.read_csv("vibration_data3.csv", encoding="ISO-8859-1")

# Merge the files (adjust based on your need)
df_combined = pd.concat([df1, df2, df3], ignore_index=True)

# Save to a new CSV
df_combined.to_csv("merged_file.csv", index=False)

print("✅ Merged successfully!")


✅ Merged successfully!
